In [1]:
#####################################################################
# Mandatory : Run this cell and restart the notebook kernel right after
#####################################################################
!pip install torch==1.11 transformers

     |████████████████████████████████| 574 kB 22.8 MB/s eta 0:00:01
     |████████████████████████████████| 197 kB 23.2 MB/s eta 0:00:01
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193002 sha256=f2ba39c5bfc065aae279bed1af2dbc61228ec33cdc462937c87bff48a815fdbb
  Stored in directory: /home/pbeuran/.cache/pip/wheels/23/a5/a8/e74bad1ceced228b6ae94dcbacc5c67df6486fd1620714e7d1
Successfully built emoji


In [2]:
%cd /content

[Errno 2] No such file or directory: '/content'
/home/pbeuran/personal_repositories/deepqa/notebooks


In [ ]:
%%bash
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
# Clone the repo content into
cd /content
rm -rf deepqa
git clone -b model https://github.com/PaulBeuran/deepqa.git

In [ ]:
%%bash
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
# Clone the repo content into
cd /content
rm -rf deepqa
git clone -b model https://github.com/PaulBeuran/deepqa.git

In [15]:
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
%cd deepqa/notebooks/

[Errno 2] No such file or directory: 'deepqa/notebooks/'
/home/pbeuran/personal_repositories/deepqa/notebooks


In [4]:
%%bash
cd ..
curl -O https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
unzip -o glove.6B.zip -d word_encoders_configs
rm glove.6B.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  822M  100  822M    0     0  4124k      0  0:03:24  0:03:24 --:--:-- 4146k


Archive:  glove.6B.zip
  inflating: word_encoders_configs/glove.6B.50d.txt  
  inflating: word_encoders_configs/glove.6B.100d.txt  
  inflating: word_encoders_configs/glove.6B.200d.txt  
  inflating: word_encoders_configs/glove.6B.300d.txt  


In [2]:
%%bash
# Dowload the SQuAD1.1 data
curl -O https://data.deepai.org/squad1.1.zip
unzip -o squad1.1.zip -d data

2022-07-30 00:36:05.538503: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-30 00:36:05.538577: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [18]:
import sys
import os
import pathlib

import json

import pandas as pd
import numpy as np
import torch

from tqdm import tqdm

USE_CPU = True
TORCH_CUDA_IS_AVAILABLE = torch.cuda.is_available()
USE_GPU = not USE_CPU and TORCH_CUDA_IS_AVAILABLE

if USE_GPU:
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
else:
    torch.set_default_tensor_type(torch.FloatTensor)

deepqa_lib_path = str(pathlib.Path(os.getcwd()).parent.parent.absolute())
sys.path.insert(0, deepqa_lib_path)

from deepqa import utils, preprocessing, tokenizer, module, model, wrapper, loss, metrics 

In [19]:
# Read train and dev data
with open("data/train-v1.1.json", "rb") as j:
    train_data = json.load(j)["data"]
with open("data/dev-v1.1.json", "rb") as j:
    dev_data = json.load(j)["data"]

In [20]:
train_tcqa = preprocessing.tabularize_squad11_data(train_data)
dev_tcqa = preprocessing.tabularize_squad11_data(dev_data)

In [21]:
glove_tokenizer = tokenizer.GLoVETokenizer(tokenize_char=False)

context_max_length = 300
query_max_length = 40
token_max_length = 40

train_tokens_cqa = glove_tokenizer.tokenize_qa_data(
    *(train_tcqa[1:]), context_max_length, query_max_length, token_max_length
)
dev_tokens_cqa = glove_tokenizer.tokenize_qa_data(
    *(dev_tcqa[1:]), context_max_length, query_max_length, token_max_length
)

train_dataset = utils.QADataset(*train_tokens_cqa, "cuda" if USE_GPU else "cpu")
dev_dataset = utils.QADataset(*dev_tokens_cqa, "cuda" if USE_GPU else "cpu")

In [22]:
char_cnn = module.CharCNN(1371, 16, [100], [5])

glove_word_encoder = module.GLoVEWordEncoder(embedding_size=200, char_encoder=None)

In [25]:
contextual_embedding_size = 100

bidaf = model.BiDAF(glove_word_encoder, contextual_embedding_size, 0.2)

In [26]:
train_loader = torch.utils.data.DataLoader(train_dataset, 64)
dev_loader = torch.utils.data.DataLoader(dev_dataset, 64)

In [27]:
bidaf_trainer = wrapper.QATrainWrapper(bidaf)
bidaf_trainer.train(train_loader, 
                    epochs=2, 
                    loss=loss.bi_cross_entropy,
                    val_data_iter=dev_loader)

Epoch 1/16 - Train - Loss: 11.396, Metrics: {'EM': 0.0, 'F1': 0.012}:   0%|          | 2/1345 [00:20<3:53:53, 10.45s/it]


KeyboardInterrupt: 

device(type='cuda', index=0)